In [1]:
import pandas as pd 
import plotly.express as px 
import scipy.stats as stats 

In [ ]:
df1 = pd.read_csv('01_Data.csv')
print(df1.shape)
df1.head(2)

# 특성 공학 (Feature Engineering) 

- **특성 공학** : 학습의 목적에 맞게 데이터를 다듬거나 학습 알고리즘을 통제하는 작업
- 대표적인 특성공학 기법 :
  - Scaling & Encoding : 숫자데이터의 스케일을 맞추거나 문자데이터를 숫자로 변환
  - Imputation : 결측값을 대치하여, 새로운 데이터가 들어올 때, 결측값이 있더라도 예측/분류
  - Cross Validation : 학습 데이터를 여러단계로 나누어 분할하여 학습
  - Hyper Parameter Tuning : 알고리즘 내 수학적 구조나, 학습이 수행되며 발생하는 구조를 통제
  - Imbalanaced Data Sampling (분류) : 데이터의 비율이 깨져 있는 경우, 한쪽의 데이터를 지우거나 생성하여 데이터의 비율을 임의로 맞춤
  - Feature Selection : 목표변수Y에 가장 영향력이 있는(회귀계수가 높은) 인자를 사용자가 지정한 만큼 선택해 사용하는 기법
    ...   

In [6]:
# 1. 데이터 핸들링 
cond1 = df1['State'].isin(['계약확정','기간만료'])
df1.loc[cond1, 'Target']  ='정상'
df1.loc[~cond1, 'Target'] ='해약'

In [9]:
# 2. 변수 선택 
X = df1[['Age','Gender','Product_Type','Amount_Month','Term','Credit_Rank']]
Y = df1['Target']

- Pipe Line : 데이터의 전처리 과정과 모델의 학습 단계를 하나의 파이썬 객체로 묶어 처리 

- Scaling
  - 1. Standard Scaling : 평균이 0 / 표준편차 1 (-> 선형회귀 기반 알고리즘) 
  - 2. Min Max Scaling  : 최대값이 1 / 최소값 0 (-> 문자데이터 / 비정형 )  
  - 3. Robust Scaling   : 중앙값 0 / IQR(사분범위) 1 (-> 이상치가 많은 데이터)
- Encoding
  - 1. Label Encoding : 각 문자 항목을 하나의 정수로 Mapping 처리
  - 2. One Hot Encoding : 문자 항목을 여러Column으로 변환하여, 데이터가 있으면 1 없으면 0 처리

In [12]:
from sklearn.pipeline import make_pipeline # 특성공학 + 학습 수행 
from sklearn.impute   import SimpleImputer # 결측값을 처리 (단순 대치) 
from sklearn.preprocessing import StandardScaler # 스케일링 기법 ( 평균 0 / 표준편차 1 )
from sklearn.preprocessing import OneHotEncoder  # 인코딩 기법 적용 
from sklearn.tree import DecisionTreeClassifier # 분류 학습을 수행 
from sklearn.model_selection import train_test_split # 학습데이터와 검증데이터 분할 

In [13]:
# random_state= : 특정 고유 정수값에 대해, 데이터가 Random하게 똑같이 추출됨 
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state=1234)

In [18]:
# 숫자 처리 파이프 구성 (결측값 - 중앙값 / 스케일링 - Standard)
numeric_pipe  = make_pipeline( SimpleImputer(strategy='median'), StandardScaler() )

In [19]:
# 문자 처리 파이프 구성 (결측값 - 최빈값 / 인코딩 - Onehot Encoding) 
category_pipe = make_pipeline( SimpleImputer(strategy='most_frequent'), OneHotEncoder() )

In [20]:
# 파이프 라인을 병렬구조로 배치 
from sklearn.compose import make_column_transformer 

In [22]:
# 각 데이터 타입을 구분 짓는 리스트 생성  
numeric_list = ['Age', 'Amount_Month', 'Term', 'Credit_Rank']
category_list = ['Gender', 'Product_Type']

In [26]:
pipe_model = make_column_transformer( (numeric_pipe, numeric_list) , 
                                      (category_pipe, category_list) )

In [29]:
# 특성공학에서 학습까지 파이프라인을 구성 
pipe_model2 = make_pipeline(pipe_model, DecisionTreeClassifier()) 

In [ ]:
pipe_model2.fit(X_train, Y_train) # 생성한 파이프라인에 데이터를 넣어 학습 

In [32]:
# 생성한 수식(모델) 파일형태로 변환
import pickle # 파이썬에서 선언된 객체를 변수로 저장 

In [33]:
pickle.dump(pipe_model2, open('model.sav', 'wb'))

# 미션 풀이

In [34]:
df1 = pd.read_csv('12_Data.csv')

In [ ]:
# "Max" 이름이 붙은 Column의 기술통계량을 확인하시오.
df1[['Max Radius', 'Max Perimeter', 'Max Area', 'Max Texture',
       'Max Smoothness', 'Max Compactness', 'Max Concavity',
       'Max Concave Points', 'Max Symmetry', 'Max Fractal Dim']].describe()

In [39]:
# "Max" 이름이 붙은 Column의 기술통계량을 확인하시오.
# 리스트 컴프리헨션 
list1 = ['a','b','c','d']
[x +'01' for x in list1 ]

['a01', 'b01', 'c01', 'd01']

In [41]:
[x for x in list1 if 'a' in x]

['a']

In [44]:
# 데이터 프레임의 컬럼 명을 담은 리스트를 변수로 선언 
list_max = df1.columns.tolist()

In [ ]:
# 데이터 컬럼 명이 있는 리스트를 리스트컴프리헨션을 활용하여 처리
 # list_max 안의 인자를 x로 받아서, Max 글자가 있다면, 리스트로 출력 
df1[[x for x in list_max if 'Max' in x]].describe()

2. Mean Radius(세포 평균 반지름)값에서 평균으로부터 표준편차 3배 이상 떨어진 데이터의 개수를 확인하시오.

In [54]:
cond1 = df1['Mean Radius'] >= df1['Mean Radius'].mean() + (df1['Mean Radius'].std() *3)
cond2 = df1['Mean Radius'] <= df1['Mean Radius'].mean() - (df1['Mean Radius'].std() *3)
df1.loc[cond1 | cond2].shape[0]

5

3. Diagnosis(진단 - 암/정상세포 여부)에 따라 Mean Radius(세포 평균 반지름)의 대표값의 차이가 있는지 가설검정을 수행하시오.
    - 가설 및 결론 작성

In [56]:
# 3-1. 각 집단의 연속형 자료의 정규성 검정 
cond1 = df1['Diagnosis']=='M'
cond2 = df1['Diagnosis']=='B'
df1_M = df1.loc[cond1]
df1_B = df1.loc[cond2]

In [58]:
print(stats.normaltest(df1_M['Mean Radius'])) # P.value < 0.05 / 대립가설 참
print(stats.normaltest(df1_B['Mean Radius'])) # P.value > 0.05 / 귀무가설 참 

NormaltestResult(statistic=9.774681423412504, pvalue=0.0075414507243749054)
NormaltestResult(statistic=0.431413656391856, pvalue=0.805971555375275)


In [59]:
# 3-2. 정규성을 따르지 않는 경우에서, 두 집단의 Mean Radius 값의 중앙값 차이를 검정 
# 귀무가설 : 두 집단의 중앙값의 차이가 없다 / 대립가설 : 두 집단의 중앙값의 차이가 있다. 
stats.ranksums(df1_M['Mean Radius'], df1_B['Mean Radius']) # P.value < 0.05 / 대립가설 참 

RanksumsResult(statistic=17.46423979127818, pvalue=2.682506544617575e-68)

4. Mean Radius(세포 평균 반지름)과 Mean Smoothness(세포 평균 곡률/매끄러움 정도)의 상관성이 있는지 가설검정을 수행하고, 상관 계수를 확인하시오. Correlation Coefficent
    - 가설 및 결론 작성 

In [60]:
# 4-1. 두 숫자 데이터가 정규분포를 따르는지 확인 
print(stats.normaltest(df1['Mean Radius'])) # P.value < 0.05 
print(stats.normaltest(df1['Mean Smoothness'])) # P.value < 0.05 

NormaltestResult(statistic=73.19882231579592, pvalue=1.2737309250385324e-16)
NormaltestResult(statistic=16.224407582610276, pvalue=0.00029985732154021345)


In [62]:
#print(stats.normaltest(df1_B['Mean Radius'])) # P.value > 0.05 
#print(stats.normaltest(df1_B['Mean Smoothness'])) # P.value < 0.05 

In [63]:
# 4-2. 정규성을 따르지 않는 숫자 데이터의 상관성 검정 
# 귀무가설 : 두 연속형 자료가 서로 상관성이 없다. / 대립가설 : 두 연속형 자료가 서로 상관성이 있다.
stats.spearmanr(df1['Mean Radius'], df1['Mean Smoothness']) # P.value < 0.05 / 대립가설참
# Corrleation : + 12.59% 

SignificanceResult(statistic=0.125935259003714, pvalue=0.002617573533346598)

In [64]:
stats.spearmanr(df1_B['Mean Radius'], df1_B['Mean Smoothness']) # P.value < 0.05 / 대립가설참
# Corrleation : - 18.66% 

SignificanceResult(statistic=-0.18656800317240865, pvalue=0.0003942533205261982)

5. 'Mean'이름이 포함된 모든 Column에 대해 정규성 검정을 실시하고, 정규분포를 따르지 않는 Column들을 모두 Histogram으로 시각화 하는 함수를 구성하시오.

In [ ]:
list_Mean = [x for x in df1.columns if 'Mean' in x]
for i in list_Mean:
    print(i +' Normal Test', stats.normaltest(df1[i])[1])

In [80]:
def non_hist():
    list_Mean = [x for x in df1.columns if 'Mean' in x]
    non_normal = [] 
    for i in list_Mean:
        p_value = stats.normaltest(df1[i])[1]
        if p_value < 0.05:
            non_normal.append(i)
    df2 = df1[non_normal].melt()
    return px.histogram(df2, x='value', color='variable')

6. 세포의 기하구조를 입력했을 때, 암/정상 세포를 판단하는 분류모델을 만들고자 한다. 'Mean'이름이 포함된 모든 Column을 X로, Diagnosis를 Y로 선언하여, X를 입력했을 때, Y를 분류하는 분류모델을 생성하시오.
    - 알고리즘은 Decision Tree를 이용
    - 학습 / 검증 데이터 비율은 75% : 25% (Random State - 1234)
    - 결측값은 중앙값으로 처리 / 스케일링은 Standard Scaling
    - 생성된 모델을 model_cell.sav로 저장

# Cross Validation 

- 교차검증 : 학습 단계에서 일반화가 더욱 잘 수행되기 위해, 학습 데이터를 여러 단계로 나누어 교차로 검증하는 기법
- Train Set 내에서 Train Set과 Validation Set을 구분하여, 반복적으로 Validation Set 바꿔 가며 학습
- K-Fold Cross Validation : 특정 K 개수 만큼 학습 데이터를 나누어 교차로 검증 (5 ~ 10)
- **Stratified K-Fold Cross Validation** : 특정 K 개수 만큼 학습 데이터를 나누는데, 목표변수의 항목간의 비율을 유지한 채로 나누어 교차로 검증  

In [84]:
X = df1[list_Mean]
Y = df1['Diagnosis']

In [85]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                            random_state=1234, test_size=0.25)

In [87]:
pipe_model = make_pipeline(SimpleImputer(strategy='median'), StandardScaler(), 
                           DecisionTreeClassifier())

In [88]:
from sklearn.model_selection import GridSearchCV # CV : Cross Validation

In [ ]:
grid_model = GridSearchCV(pipe_model, param_grid={}, cv=5, scoring='f1' )
# cv = : 몇 개의 Fold 를 지정 할 것인가 / scoring = : 어떤 지표로 평가 할 것인가 
grid_model.fit(X_train, Y_train)
# 성능이 가장 좋은 모델을 선정 (Cross Validation에서 기본 성능 지표 -> Acc(분류)/ R2(결정계수)) 
best_model = grid_model.best_estimator_

# Hyper Parameter Tuning 

- Hyper Parameter : Model을 학습 시키기 전, 학습 알고리즘 내 세팅되어 있는 매개변수
- 학습 알고리즘 내 지정된 매개변수를 사용자가 조절하여, Overfitting을 방지, 학습/일반화 성능을 높이는 기법 (알고리즘 내 존재하는 고유의 Hyper Parameter를 사전에 알고 있어야 함)
  - Random Search : 매개변수의 조합을 임의로 설정하여 학습 
  - Grid Search : 사용자가 지정한 범위내 매개변수의 모든 조합에 대해 학습  
  - Bayesian Optimization : 베이지안 최적화 기법을 이용해, 불확실성이 가장 낮은 매개변수를 선택해 학습 

In [ ]:
hypermeter = {'decisiontreeclassifier__max_depth':range(5,20), # 15가지 경우의 수 
             'decisiontreeclassifier__min_samples_split':range(5,10)}  # 5가지의 경우의수 
# 하이퍼파라미터 : 15번 x 5번 = 75개 
# Scikit Learn 라이브러리는 CPU를 이용해 연산을 수행 
grid_model = GridSearchCV(pipe_model, param_grid=hypermeter, 
                          cv=3, scoring='f1', n_jobs=-1 ) # 75 x 3 => 225번 학습이 수행 
grid_model.fit(X_train, Y_train)

In [98]:
best_model = grid_model.best_estimator_ # 모든 경우의 수 중 가장 성능이 좋은 모델 도출 
pickle.dump(best_model, open('model_cell.sav', 'wb')) # 가장 성능이 좋은 모델을 저장 

7. 앞서 생성한 모델의 아래의 지표를 이용해 성능 평가를 수행하시오.

In [99]:
from sklearn.metrics import classification_report

In [100]:
Y_train_pred = best_model.predict(X_train)
Y_test_pred = best_model.predict(X_test)

In [102]:
print(classification_report(Y_train, Y_train_pred))
print(classification_report(Y_test,  Y_test_pred))

              precision    recall  f1-score   support

           B       1.00      0.99      0.99       269
           M       0.98      0.99      0.99       157

    accuracy                           0.99       426
   macro avg       0.99      0.99      0.99       426
weighted avg       0.99      0.99      0.99       426

              precision    recall  f1-score   support

           B       0.90      0.94      0.92        88
           M       0.90      0.84      0.87        55

    accuracy                           0.90       143
   macro avg       0.90      0.89      0.90       143
weighted avg       0.90      0.90      0.90       143

